#### [ TEST ]


[1] 모듈 로딩 및 데이터 준비<hr>

In [5]:
## [1-1] 모듈 로딩
import cv2                           # 컴퓨터비젼 모듈
import numpy as np                   # 이미지 데이터 저장 모듈
import matplotlib.pyplot as plt      # 시각화 모듈
import os                            # 파일, 폴더, 경로 관련 모듈

In [6]:
## [1-2] 데이터 준비
FILE_CSV = '../image/cat_dog.csv'

In [7]:
if os.path.exists(FILE_CSV):
    print(f'{FILE_CSV} Dataset 파일이 없습니다.')

../image/cat_dog.csv Dataset 파일이 없습니다.


[2] 이미지 데이터 로딩 <hr>

In [8]:
## [2-1] 피쳐와 타겟 분리
import pandas as pd 

catdogDF=pd.read_csv(FILE_CSV, header=None)
catdogDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Columns: 2501 entries, 0 to 2500
dtypes: int64(2500), object(1)
memory usage: 6.5+ MB


[3] 학습 준비<hr>
- 피쳐와 타겟 분리
- 피쳐 스케일링
- 학습용, 테스트용 데이터셋 분리

In [9]:
## [3-1] 피쳐 & 타겟 분리
featureDF = catdogDF[catdogDF.columns[1:]]
targetSR = catdogDF[catdogDF.columns[0]]

print(f'featureDF : {featureDF.shape}   targetSR : {targetSR.shape}')

featureDF : (342, 2500)   targetSR : (342,)


In [10]:
## [3-2] 피쳐 정규화
featureDF = featureDF/255.
featureDF.describe()

,1,2,3,4,5,6,7,8,9,10,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
count,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,...,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000
mean,0.560681,0.563834,0.570669,0.571895,0.581252,0.588682,0.593361,0.590517,0.594806,0.600986,...,0.593831,0.592719,0.584188,0.577720,0.581963,0.581126,0.573008,0.572870,0.571976,0.569155
std,0.319030,0.321847,0.316375,0.318908,0.313529,0.303386,0.304241,0.305072,0.304310,0.304861,...,0.284963,0.277574,0.285276,0.284040,0.281684,0.277189,0.279249,0.279823,0.278049,0.281997
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.277451,0.251961,0.301961,0.288235,0.307843,0.309804,0.321569,0.333333,0.342157,0.352941,...,0.393137,0.392157,0.366667,0.352941,0.381373,0.377451,0.361765,0.352941,0.368627,0.342157
50%,0.600000,0.603922,0.607843,0.607843,0.617647,0.633333,0.633333,0.637255,0.656863,0.660784,...,0.590196,0.588235,0.588235,0.568627,0.586275,0.578431,0.580392,0.584314,0.562745,0.541176
75%,0.850980,0.862745,0.853922,0.854902,0.857843,0.850980,0.850980,0.853922,0.849020,0.858824,...,0.845098,0.839216,0.839216,0.826471,0.822549,0.815686,0.814706,0.818627,0.819608,0.822549
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
## [3-3] 학습용, 테스트용 데이터셋 분리
from sklearn.model_selection import train_test_split

x_train ,x_test, y_train, y_test = train_test_split(featureDF, targetSR,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=targetSR)

In [12]:
print(f'x_train : {x_train.shape}   y_train : {y_train.shape}')
print(f'x_test : {x_test.shape}   y_test : {y_test.shape}')

x_train : (273, 2500)   y_train : (273,)
x_test : (69, 2500)   y_test : (69,)


[4] 학습 진행<hr>
- 최적 모델 체크
- 기준선 설정 
- 학습 진행

In [13]:
## [4-1] 최적 모델 체크
from sklearn.utils.discovery import *
from sklearn.metrics import *
import warnings

# {“classifier”, “regressor”, “cluster”, “transformer”} 
rets=all_estimators(type_filter='classifier')

In [14]:
resultList=[]
for name, estimator_ in rets:
    try:
        model=estimator_()
        if 'Logistic' in name or 'SGD' in name or 'MLP' in name:
            model.set_params(max_iter=10000)
        if 'SV' in name:
            model.set_params(max_iter=100000, dual='auto')   
 
        model.fit(x_train, y_train)

        trainScore= model.score(x_train, y_train)
        testScore = model.score(x_test, y_test)

        resultList.append((name, round(trainScore, 3), round(testScore, 3)))
    except Exception:
        pass

c:\Users\KDT17\anaconda3\envs\ML_CV\lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\KDT17\anaconda3\envs\ML_CV\lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\KDT17\anaconda3\envs\ML_CV\lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\KDT17\anaconda3\envs\ML_CV\lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\KDT17\anaconda3\envs\ML_CV\lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\KDT17\anaconda3\envs\ML_CV\lib\site-packages\sklearn\semi_supervised\_label_

In [15]:
sorted(resultList, key=lambda x : x[2], reverse=True)

[('ExtraTreesClassifier', 0.996, 0.667),
 ('RandomForestClassifier', 0.996, 0.652),
 ('LogisticRegression', 0.996, 0.623),
 ('GradientBoostingClassifier', 0.996, 0.609),
 ('LogisticRegressionCV', 0.919, 0.609),
 ('BaggingClassifier', 0.971, 0.594),
 ('HistGradientBoostingClassifier', 0.996, 0.594),
 ('MLPClassifier', 0.978, 0.58),
 ('CalibratedClassifierCV', 0.791, 0.565),
 ('RidgeClassifierCV', 0.993, 0.565),
 ('PassiveAggressiveClassifier', 0.908, 0.551),
 ('Perceptron', 0.93, 0.536),
 ('RidgeClassifier', 0.996, 0.536),
 ('AdaBoostClassifier', 0.886, 0.522),
 ('KNeighborsClassifier', 0.623, 0.522),
 ('LinearDiscriminantAnalysis', 0.993, 0.522),
 ('LinearSVC', 0.996, 0.522),
 ('QuadraticDiscriminantAnalysis', 0.996, 0.522),
 ('ExtraTreeClassifier', 0.996, 0.507),
 ('DummyClassifier', 0.484, 0.493),
 ('BernoulliNB', 0.524, 0.478),
 ('ComplementNB', 0.524, 0.464),
 ('GaussianNB', 0.498, 0.464),
 ('LabelPropagation', 0.996, 0.464),
 ('LabelSpreading', 0.996, 0.464),
 ('DecisionTreeClassi

In [16]:
## [4-1] 최적 모델 체크
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train , y_train)

RandomForestClassifier()

In [17]:
from sklearn.metrics import classification_report

trainScore= model.score(x_train, y_train)
testScore = model.score(x_test, y_test)
print('[TRAIN]' , trainScore , '[TEST]' , testScore)

pre = model.predict(x_test)
resultDict=classification_report(y_test, pre)
print(resultDict)

[TRAIN] 0.9963369963369964 [TEST] 0.6956521739130435
              precision    recall  f1-score   support

         Dog       0.73      0.50      0.59        16
         cat       0.67      0.97      0.80        34
         dog       0.78      0.37      0.50        19

    accuracy                           0.70        69
   macro avg       0.73      0.61      0.63        69
weighted avg       0.71      0.70      0.67        69



[5] 학습 및 튜닝 진행 <hr>